# How to stream LLM tokens from specific nodes
- https://langchain-ai.github.io/langgraph/how-tos/streaming-specific-nodes/
- A common use case of stream LLM tokens is only stream them from specific nodes. To do so, you can use `stream_mode="messages"` and filter the outputs by the `langgraph_node` field in the streamed metadata

In [5]:
from typing import TypedDict
from langgraph.graph import START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")


class State(TypedDict):
    topic: str
    joke: str
    poem: str


def write_joke(state: State):
    topic = state["topic"]
    joke_response = model.invoke(
        [{"role": "user", "content": f"Write a joke about {topic}"}]
    )
    return {"joke": joke_response.content}


def write_poem(state: State):
    topic = state["topic"]
    poem_response = model.invoke(
        [{"role": "user", "content": f"Write a short poem about {topic}"}]
    )
    return {"poem": poem_response.content}


graph = (
    StateGraph(State)
    .add_node(write_joke)
    .add_node(write_poem)
    # write both the joke and the poem concurrently
    .add_edge(START, "write_joke")
    .add_edge(START, "write_poem")
    .compile()
)

In [6]:
for msg, metadata in graph.stream(
    {"topic": "cats"},
    stream_mode="messages",
):
    if msg.content and metadata["langgraph_node"] == "write_poem":
        print(msg.content, end="|", flush=True)

In| quiet| shadows|,| soft| and| sleek|,|  
|A| whisper| of| grace|,| a| playful| peek|.|  
|With| bound|less| leaps| and| gentle| p|urr|s|,|  
|In| sun|lit| spots|,| their| magic| st|irs|.|  

|Vel|vet| paws| on| midnight|’s| pro|wl|,|  
|A| flick| of| tail|,| a| sudden| grow|l|.|  
|They| dance| through| dreams| with| silent| po|ise|,|  
|Our| furry| friends|,| our| heart|’s| soft| joys|.|  

|In| every| n|uzzle|,| every| glance|,|  
|They| weave| their| spells|,| they| make| us| dance|.|  
|With| feline| charm|,| they| steal| our| days|,|  
|In| their| sweet| world|,| we| lose| our| ways|.|  |